In [2]:
import pprint
from gensim import corpora
from gensim import models
from gensim import similarities
import pandas as pd
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

C:\Users\muham\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
## import clean subreddit data

In [4]:
reddit_data = pd.read_csv("data/2019-11-02_reddit-data-learnmath_scrubbed.csv")

In [5]:
reddit_data.head()

,Unnamed: 0,discussion_topic_title,discussion_topic_message,upvotes,thread_ref_link,combined
0,0,list of websites ebooks downloads etc for mobi...,feel free to suggest more videos all level...,547,https://www.reddit.com/r/learnmath/comments/8p...,list of websites ebooks downloads etc for mobi...
1,1,reintroducing rorbibliographies,hi im the new mod for orrorbibliographies i r...,141,https://www.reddit.com/r/learnmath/comments/ak...,reintroducing rorbibliographies hi im the new...
2,2,man has walked into a casino with 1000 to play...,i translated this problem from other language ...,78,https://www.reddit.com/r/learnmath/comments/dq...,man has walked into a casino with 1000 to play...
3,3,i guys my post was removed on orrormath so i a...,to give a little bit of my background about my...,22,https://www.reddit.com/r/learnmath/comments/dq...,i guys my post was removed on orrormath so i a...
4,4,college how to avoid being daunted by the work...,hi i am a late learner of collegelevel math an...,12,https://www.reddit.com/r/learnmath/comments/dq...,college how to avoid being daunted by the work...


In [6]:
reddit_data["discussion_topic_title"][0]

'list of websites ebooks downloads etc for mobile users and people too lazy to read the sidebar'

In [7]:
reddit_data = reddit_data["discussion_topic_title"]

In [9]:
# Convert to a list 
corpus_text = list()
for row in range(reddit_data.shape[0]):
    temp = reddit_data.iloc[row]
    corpus_text.append(temp)

In [12]:
stop_words = stopwords.words('english')

In [21]:
a = [1 , 2 ,3 ,4]

In [25]:
list(map(lambda x: x**2, a))

[1, 4, 9, 16]

In [26]:
corpus_text[0]

'list of websites ebooks downloads etc for mobile users and people too lazy to read the sidebar'

In [28]:
from nltk.stem import PorterStemmer

In [30]:
# Convert a string into a list of words, to generate a list of lists
# clean words and remove stop words

processed_corpus = list()
porter=PorterStemmer()

for text in corpus_text:
    
    token_words=word_tokenize(text)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
        
    # tokenize it 
    tokenized_list = word_tokenize(text)
    
    # convert to lower case
    tokenized_list = [w.lower() for w in stem_sentence]
    
    # get the alphabetic words
    words = [word for word in tokenized_list if word.isalpha()]
    
    # get rid of stop words 
    words = [w for w in words if not w in stop_words]
    
    # stemming
    lambda x: x
            
    processed_corpus.append(words)

In [31]:
processed_corpus[0]

['list',
 'websit',
 'ebook',
 'download',
 'etc',
 'mobil',
 'user',
 'peopl',
 'lazi',
 'read',
 'sidebar']

In [32]:
# create dictionary
dictionary_reddit = corpora.Dictionary(processed_corpus)
print(dictionary_reddit)

Dictionary(1452 unique tokens: ['download', 'ebook', 'etc', 'lazi', 'list']...)


In [15]:
# create dictionary
dictionary_reddit = corpora.Dictionary(processed_corpus)
print(dictionary_reddit)

Dictionary(1829 unique tokens: ['downloads', 'ebooks', 'etc', 'lazy', 'list']...)


In [33]:
# convert to a bag of words reprensentation
bow_corpus_reddit = [dictionary_reddit.doc2bow(text) for text in processed_corpus]
#pprint.pprint(bow_corpus_reddit)

In [34]:
# train the model
tfidf_reddit = models.TfidfModel(bow_corpus_reddit)

In [35]:
# similarities indexing
index_reddit = similarities.SparseMatrixSimilarity(tfidf_reddit[bow_corpus_reddit], num_features = 1829)

In [38]:
# test a query
query_document = 'how to do differential equation'
token_words=word_tokenize(query_document)
stem_sentence=[]
for word in token_words:
    stem_sentence.append(porter.stem(word))
# bag of words representation of the query
query_bow = dictionary_reddit.doc2bow(stem_sentence)

In [39]:
# create the similarities scores
sims_reddit = index_reddit[tfidf_reddit[query_bow]]

In [40]:
# put scores in a dict
sim_scores = dict()
for document_number, score in sorted(enumerate(sims_reddit)):
    sim_scores[document_number] = score

In [41]:
# sort the scores
sorted_sim_scores = sorted(sim_scores.items(), key=lambda kv: kv[1], reverse = True)

In [46]:
reddit_data[]

'what is an equation'

In [42]:
sorted_sim_scores[:10]

[(402, 1.0),
 (460, 1.0),
 (364, 0.8363477),
 (102, 0.63148826),
 (16, 0.62982845),
 (754, 0.56039983),
 (403, 0.5006765),
 (147, 0.49865982),
 (226, 0.49456146),
 (236, 0.46900314)]

In [134]:
sorted_sim_scores[:10]

[(460, 0.99999994),
 (102, 0.66516),
 (754, 0.6157832),
 (402, 0.5454679),
 (209, 0.4805221),
 (364, 0.4607327),
 (412, 0.42027405),
 (878, 0.3622085),
 (376, 0.35159275),
 (933, 0.34474713)]

In [135]:
reddit_data[460]

'differential equation'

In [136]:
reddit_data[102]

'what is an equation'

In [137]:
reddit_data[754]

'non linear differential equation'

In [138]:
reddit_data[402]

'differential equations'

In [139]:
reddit_data[209]

'advanced calculus general solution of a differential equation'

**This cell is to remove words with freq = 1, MIGHT SKIP**

In [51]:
# Count word frequencies
frequency = defaultdict(int)
for text in processed_corpus:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once (remove for now)
#processed_corpus = token for token in texts_clean if frequency[token] > 1
processed_corpus = [[token for token in text if frequency[token] > 1] for text in processed_corpus]

In [109]:
processed_corpus[0] # will be different from above if have removed freq = 1 words

['list',
 'websites',
 'ebooks',
 'downloads',
 'etc',
 'mobile',
 'users',
 'people',
 'lazy',
 'read',
 'sidebar']